# Exploratory Analysis for LLM Travel Chatbot

- This notebook contains exploratory analysis to inform the development of a large language model (LLM) chatbot designed to assist users with questions about their personal travel itineraries.  
   
   
---
   
## Objectives
- Extract gmail from dafamtripbot@gmail.com
- Use LLM to extract flight info from text 


In [ ]:
import base64
import json
import os
from datetime import datetime, date, time
from dotenv import load_dotenv
from pathlib import Path
from typing import Literal, List, Optional

from bs4 import BeautifulSoup  
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field



In [ ]:
load_dotenv(dotenv_path=Path.home() / ".env")

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [ ]:
class Passenger(BaseModel):
    first_name: str
    last_name: str

class FlightDetails(BaseModel):
    flight_number: str
    airline_name: str
    departure_date: Optional[date] = None
    departure_time: Optional[time] = None
    arrival_date: Optional[datetime] = None
    arrival_time: Optional[str] = None
    origin: str
    destination: str
    passengers: List[Passenger]

class FlightManifest(BaseModel):
    flights: List[FlightDetails]
    
flight_parser = PydanticOutputParser(pydantic_object=FlightManifest)


In [ ]:
def get_gmail_service():
    creds = None
    # Load saved credentials if available
    if Path('token.json').exists():
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    # If no valid credentials, run OAuth flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Get credentials from environment variables
            client_config = {
                "installed": {
                    "client_id": os.getenv("TRAVELBOT_GMAIL_CLIENT_ID"),
                    "client_secret": os.getenv("TRAVELBOT_GMAIL_CLIENT_SECRET"),
                    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                    "token_uri": "https://oauth2.googleapis.com/token",
                    "redirect_uris": ["http://localhost"]
                }
            }

            flow = InstalledAppFlow.from_client_config(client_config, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

In [ ]:
def get_latest_email_id():
    service = get_gmail_service()
    # Get list of messages
    results = service.users().messages().list(userId='me', maxResults=1).execute()
    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    # Get the message details
    msg_id = messages[0]['id']
    return msg_id

 

In [ ]:
def extract_email_as_json(service, message_id):
    """Extracts email metadata and body content as JSON from Gmail API using message ID."""
    msg = service.users().messages().get(userId='me', id=message_id, format='full').execute()
    
    payload = msg.get("payload", {})
    headers = payload.get("headers", [])

    def get_header(name):
        return next((h['value'] for h in headers if h['name'].lower() == name.lower()), None)

    # Extract headers
    email_data = {
        "from": get_header("From"),
        "to": get_header("To"),
        "date": get_header("Date"),
        "subject": get_header("Subject"),
        "message_id": message_id,
        "body": None  # populated below
    }

    # Extract body (prefer text/plain over text/html)
    def extract_body(payload):
        if payload.get("mimeType") == "text/plain":
            return base64.urlsafe_b64decode(payload["body"].get("data", "")).decode("utf-8", errors="ignore")
        elif payload.get("mimeType") == "text/html":
            html = base64.urlsafe_b64decode(payload["body"].get("data", "")).decode("utf-8", errors="ignore")
            return BeautifulSoup(html, "html.parser").get_text()
        elif "parts" in payload:
            for part in payload["parts"]:
                body = extract_body(part)
                if body:
                    return body
        return None

    email_data["body"] = extract_body(payload)

    return email_data

In [ ]:
msg_id = get_latest_email_id()
data = extract_email_as_json(get_gmail_service(), msg_id)

In [ ]:
flight_email = data.get('body', '')

In [ ]:
# Using environment variable to authenticate 
client = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
extract_flight = """I am building a trip itinerary for a family vacation. The following text in triple quotes contains flight information from the email confirmation I received from the airline.  There can be multiple passengers and flights in a single email confirmation. Extract the following relevant passenger and flight information.

The following pieces of information should be collected for each passenger that is traveling. Remember, there can be multiple passengers on each flight
- first_name 
- last_name 

The following pieces of information should be collected for each flight. There are usually multiple flights per email. 
- departure_date
- departure_time
- arrival_date
- arrival_time 
- origin
- destination
- flight_number
- airline_name

 ```{email}```

{format_instructions}
"""



extract_flight_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract structured flight and passenger information from the text and convert it to JSON using ISO 8601 format for all datetime fields."),
    ("human", extract_flight)
])

flight_chain = extract_flight_prompt | client | flight_parser


In [ ]:
flight_chainparams = {'email': flight_email,
                      'format_instructions': flight_parser.get_format_instructions()}
flight_response = flight_chain.invoke(flight_chainparams)

In [ ]:
flight_response.dict()